# Finding Similar Songs on Spotify - Part 2 - Siamese Networks

In this tutorial I will demonstrate how to apply machine learning to search for similar songs on Spotify.


## Tutorial Overview

1. Loading data
2. Preprocess data
3. Define Model
4. Fit Model
5. Evaluate Model



## Requiremnts

Install the following dependencies to run this tutorial:

In [1]:
%load_ext autoreload

%autoreload 2

# visualization
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

# numeric and scientific processing
import numpy as np
import pandas as pd

# misc
import os
import progressbar

import tutorial_functions as tut_func

**Spotipy**

Spotipy is a thin client library for the Spotify Web API.

https://github.com/plamere/spotipy

In [2]:
import spotipy
import spotipy.util as util

TODO: describe how to get client credentials

In [3]:
os.environ["SPOTIPY_CLIENT_ID"]     = "8a7fffc37b6c44e6b7bc344c3295034c"
os.environ["SPOTIPY_CLIENT_SECRET"] = "f19dd914ba58408c9407dd6479b23812"

If you get the following message:

    User authentication requires interaction with your
    web browser. Once you enter your credentials and
    give authorization, you will be redirected to
    a url.  Paste that url you were directed to to
    complete the authorization.

    Opened https://accounts.spotify.com/authorize?scope=playlist-modify-public&redirect_uri=ht...
    

You need to authenticate your browser session. Follow the link and log in to Spotify. After login, you will be redirected to http://localhost/?code=... Copy the entire URL and paste it to the prompted textbox.

In [4]:
token = util.prompt_for_user_token("slychief", 
                                   "playlist-modify-public", 
                                   redirect_uri="http://localhost/")

sp = spotipy.Spotify(auth=token)

# Loading Data

Before we can train our models we first have to get some data.

## Download Echonest Features from Spotify

We use spotipy to access the Spotify API and to download metadata and audio features of Spotify tracks. The following list provides a selection of Spotify playlists of various music genres.

In [5]:
playlists = [
    
     {"name": "clubbeats",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DXbX3zSzB4MO0"},
     {"name": "softpop",      "uri": "spotify:user:spotify:playlist:37i9dQZF1DWTwnEm1IYyoj"},
     {"name": "electropop",   "uri": "spotify:user:spotify:playlist:37i9dQZF1DX4uPi2roRUwU"},
     {"name": "rockclassics", "uri": "spotify:user:spotify:playlist:37i9dQZF1DWXRqgorJj26U"},
     {"name": "rockhymns",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DX4vth7idTQch"},
     {"name": "soft_rock",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DX6xOPeSOGone"},
     {"name": "metalcore",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DWXIcbzpLauPS"}, 
     {"name": "metal",        "uri": "spotify:user:spotify:playlist:37i9dQZF1DWWOaP4H0w5b0"},
     {"name": "classic_metal","uri": "spotify:user:spotify:playlist:37i9dQZF1DX2LTcinqsO68"},
     {"name": "grunge",       "uri": "spotify:user:spotify:playlist:37i9dQZF1DX11ghcIxjcjE"},
     {"name": "hiphop",       "uri": "spotify:user:spotify:playlist:37i9dQZF1DWVdgXTbYm2r0"},
     {"name": "poppunk",      "uri": "spotify:user:spotify:playlist:37i9dQZF1DXa9wYJr1oMFq"},
     {"name": "classic",      "uri": "spotify:user:spotify:playlist:37i9dQZF1DXcN1fAVSf7CR"}
    
]

### Caching with joblib

We will use caching to locally store retrieved data. This is on the one hand a requirement of the API and on the other it speeds up processing when we reload the notebook. *joblib* is a convenient library which simplifies caching.

*Update the cachdir to an appropriate path in the following cell*

In [6]:
from joblib import Memory

memory = Memory(cachedir='/home/schindler/tmp/spotify/', verbose=0)

In [7]:
@memory.cache
def get_spotify_data(track_id):
    
    # meta-data
    track_metadata      = sp.track(track_id)
    album_metadata      = sp.album(track_metadata["album"]["id"])
    artist_metadata     = sp.artist(track_metadata["artists"][0]["id"])
    
    # feature-data
    sequential_features = sp.audio_analysis(track_id)
    trackbased_features = sp.audio_features([track_id])
    
    return track_metadata, album_metadata, artist_metadata, sequential_features, trackbased_features

In [10]:
# Get Playlist meta-data
playlists = tut_func.get_playlist_metadata(sp, playlists)

# Get track-ids of all playlist entries
playlists = tut_func.get_track_ids(sp, playlists)

num_tracks_total = np.sum([playlist["num_tracks"] for playlist in playlists])

# Fetch data and features from Spotify
pbar = progressbar.ProgressBar(max_value=num_tracks_total)
pbar.start()

raw_track_data      = []
processed_track_ids = []

for playlist in playlists:

    for track_id in playlist["track_ids"]:

        try:
            # avoid duplicates in the data-set
            if track_id not in processed_track_ids:

                # retrieve data from Spotify
                spotify_data = get_spotify_data(track_id)

                raw_track_data.append([playlist["name"], spotify_data])
                processed_track_ids.append(track_id)

        except Exception as e:
            print e

        pbar.update(len(raw_track_data))

 62% (606 of 970) |#################################################################################################################                                                                     | Elapsed Time: 0:00:25 ETA: 0:00:19

Unterminated string starting at: line 1 column 65530 (char 65529)


 97% (942 of 970) |################################################################################################################################################################################      | Elapsed Time: 0:00:40 ETA: 0:00:01

# Aggregate data

Currently we only have a list of raw data-objects retrieved from the Spotify API. We need to transform this information to a more structured format.

In [12]:
# Aggregate Meta-data
metadata = tut_func.aggregate_metadata(raw_track_data)

# Aggregate Feature-data
feature_data = tut_func.aggregate_featuredata(raw_track_data, metadata)

# standardize sequential_features
feature_data -= feature_data.mean(axis=0)
feature_data /= feature_data.std(axis=0)

# Calculate Similarities

In [134]:
def create_pairs(feature_data, metadata, num_pairs_per_track):
    
    data_pairs = []
    labels     = []
    
    for row_id, q_track in metadata.sample(frac=1).iterrows():
        
        for _ in range(num_pairs_per_track):
            
            # search similar and dissimilar examples
            pos_example = metadata[metadata.playlist == q_track.playlist].sample(1)
            neg_example = metadata[metadata.playlist != q_track.playlist].sample(1)

            # create feature pairs
            data_pairs.append([feature_data[[row_id]][0], feature_data[[pos_example.index]][0]])
            labels.append(1)

            data_pairs.append([feature_data[[row_id]][0], feature_data[[neg_example.index]][0]])
            labels.append(0)

    return np.array(data_pairs), np.array(labels)

In [202]:
data_pairs, labels = create_pairs(feature_data, metadata, 10)

data_pairs.shape

(18880, 2, 69)

In [203]:
from keras.layers import Input, Lambda
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.optimizers import Nadam, SGD
from keras.regularizers import l2, l1
from keras import backend as K
from keras.layers.merge import concatenate

In [241]:
def create_siamese_network(input_dim):

    input_left  = Input(shape=input_dim)
    input_right = Input(shape=input_dim)

    layers_left  = Dense(100, activation="selu")(input_left)
    layers_left  = Dense(100, activation="selu")(layers_left)
    layers_left  = Dense(100, activation="selu")(layers_left)
    layers_right = Dense(100, activation="selu")(input_right)
    layers_right = Dense(100, activation="selu")(layers_right)
    layers_right = Dense(100, activation="selu")(layers_right)

    L1_distance = lambda x: K.abs(x[0]-x[1])

    distance = Lambda(L1_distance,
                      output_shape=lambda x: x[0])([layers_left, layers_right])

    prediction = Dense(100, activation="elu")(distance)

    prediction = Dense(1, activation="sigmoid")(prediction)

    model = Model([input_left, input_right], prediction)

    # train
    rms = Nadam(lr=0.001)
    model.compile(loss="mean_squared_error", optimizer=rms, metrics=["mean_squared_error", "accuracy"])
    
    return model

In [242]:
model = create_siamese_network(data_pairs[:,0].shape[1:])

In [207]:
model.fit([data_pairs[:, 0], data_pairs[:, 1]], labels, batch_size=24, verbose=1, epochs=25)

Epoch 1/25
18880/18880 [==============================] - 0s - loss: 0.1481 - mean_squared_error: 0.1481 - acc: 0.7882     
Epoch 2/25
18880/18880 [==============================] - 0s - loss: 0.1038 - mean_squared_error: 0.1038 - acc: 0.8584     
Epoch 3/25
18880/18880 [==============================] - 0s - loss: 0.0870 - mean_squared_error: 0.0870 - acc: 0.8837     
Epoch 4/25
18880/18880 [==============================] - 0s - loss: 0.0744 - mean_squared_error: 0.0744 - acc: 0.9015     
Epoch 5/25
18880/18880 [==============================] - 0s - loss: 0.0646 - mean_squared_error: 0.0646 - acc: 0.9163     
Epoch 6/25
18880/18880 [==============================] - 0s - loss: 0.0582 - mean_squared_error: 0.0582 - acc: 0.9236     
Epoch 7/25
18880/18880 [==============================] - 0s - loss: 0.0512 - mean_squared_error: 0.0512 - acc: 0.9323     
Epoch 8/25
18880/18880 [==============================] - 0s - loss: 0.0462 - mean_squared_error: 0.0462 - acc: 0.9404     
Epoch 9/

In [154]:
def similar(query_idx):
    res = [model.predict([feature_data[[query_idx]], feature_data[[i]]]) for i in range(feature_data.shape[0])]

    res = np.array(res)
    res = res.reshape(res.shape[0])

    si = np.argsort(res)[::-1]

    display_cols = ["artist_name", "title", "album_name", "year", "playlist"]
    
    print metadata.iloc[query_idx]

    return metadata.loc[si, display_cols][:10]

In [208]:
similar(381)

track_id                                  4CJVkjo5WpmUAKp3R44LNb
artist_name                                       Lynyrd Skynyrd
title                                         Sweet Home Alabama
album_name                                        Second Helping
label                   Digital Distribution Trinidad and Tobago
duration                                                  281147
popularity                                                    77
year                                                        1974
genres         [album rock, blues-rock, classic rock, country...
playlist                                            rockclassics
Name: 381, dtype: object


,artist_name,title,album_name,year,playlist
549,Counting Crows,Big Yellow Taxi,Films About Ghosts (The Best Of Counting Crows...,2003,soft_rock
328,Creedence Clearwater Revival,Down On The Corner,Willy And The Poor Boys (40th Anniversary Edit...,1969,rockclassics
267,Bob Seger,Ramblin' Gamblin' Man,Ramblin' Gamblin' Man,1969,rockclassics
323,The Rolling Stones,Brown Sugar - Remastered,Sticky Fingers (Deluxe),1971,rockclassics
270,AC/DC,Back In Black,Back In Black,1980,rockclassics
339,Lynyrd Skynyrd,Gimme Three Steps,(Pronounced 'Leh-'Nérd 'Skin-'Nérd) [Expanded ...,1973,rockclassics
367,Steppenwolf,Magic Carpet Ride,Born To Be Wild: A Retrospective,1991,rockclassics
546,Keane,Everybody's Changing,Hopes and Fears (Deluxe Edition),2009,soft_rock
277,Eagles,Life In The Fast Lane,Hotel California (Remastered),1976,rockclassics
383,Aerosmith,Dream On,Aerosmith,1973,rockclassics


# Evaluate

In [215]:
def evaluate(similarity_function, cut_off):

    global dist
    
    all_precisions = []

    for idx in metadata.index.values:

        dist           = similarity_function(feature_data, feature_data[[idx]])
        dist           = np.array(dist).reshape(len(dist))
        similar_tracks = metadata.loc[np.argsort(dist)[::-1][:cut_off]]
        same_label     = similar_tracks["playlist"] == metadata.loc[idx, "playlist"]
        precision      = same_label.sum() / float(cut_off)
        all_precisions.append(precision)

    all_precisions = np.array(all_precisions)

    return all_precisions.mean()

In [216]:
evaluate(lambda x,y: [model.predict([x[[i]], y]) for i in range(feature_data.shape[0])], 20)

0.82388771186440679

In [231]:
playlist_names = [pl["name"] for pl in playlists]

playlist_similarities = pd.DataFrame(np.zeros((len(playlist_names),len(playlist_names))), 
                                     index   = playlist_names, 
                                     columns = playlist_names)

In [232]:
sim = [[["clubbeats",    "electropop"],    0.8],
       [["clubbeats",    "softpop"],      0.4],
       [["electropop",   "hiphop"],      0.4],
       [["softpop",      "soft_rock"],      0.2],
       [["softpop",      "electropop"],      0.4],
       [["softpop",    "hiphop"],      0.1],
       [["rockclassics",    "rockhymns"],      0.7],
       [["soft_rock",    "rockclassics"],      0.3],
       [["soft_rock",    "rockhymns"],      0.3],
       [["metalcore",    "metal"],      0.7],
       [["metalcore",    "classic_metal"],      0.6],
       [["metal",    "classic_metal"],      0.8],
       [["classic_metal",    "grunge"],      0.5],
       [["metal",    "grunge"],      0.5],
       [["rockhymns",    "grunge"],      0.2],
       [["poppunk",    "metal"],      0.6],
       [["poppunk",    "classic_metal"],      0.4],
       [["poppunk",    "rockhymns"],      0.5],
       [["poppunk",    "rockclassics"],      0.4]
     ]

# self-similarity
for i in range(len(playlist_names)):
    for j in range(len(playlist_names)):
        if i == j:
            playlist_similarities.iloc[i,j] = 1.0

for s in sim:
    playlist_similarities.loc[s[0][0],s[0][1]] = s[1]
    playlist_similarities.loc[s[0][1],s[0][0]] = s[1]

playlist_similarities

,clubbeats,softpop,electropop,rockclassics,rockhymns,soft_rock,metalcore,metal,classic_metal,grunge,hiphop,poppunk,classic
clubbeats,1.0,0.4,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
softpop,0.4,1.0,0.4,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.1,0.0,0.0
electropop,0.8,0.4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0
rockclassics,0.0,0.0,0.0,1.0,0.7,0.3,0.0,0.0,0.0,0.0,0.0,0.4,0.0
rockhymns,0.0,0.0,0.0,0.7,1.0,0.3,0.0,0.0,0.0,0.2,0.0,0.5,0.0
soft_rock,0.0,0.2,0.0,0.3,0.3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
metalcore,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.7,0.6,0.0,0.0,0.0,0.0
metal,0.0,0.0,0.0,0.0,0.0,0.0,0.7,1.0,0.8,0.5,0.0,0.6,0.0
classic_metal,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.8,1.0,0.5,0.0,0.4,0.0
grunge,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.5,0.5,1.0,0.0,0.0,0.0


In [235]:
def create_pairs_with_sims(feature_data, metadata, num_pairs_per_track, playlist_similarities):
    
    data_pairs = []
    labels     = []
    
    for row_id, q_track in metadata.sample(frac=1).iterrows():
        
        for _ in range(num_pairs_per_track):
            
            # search similar and dissimilar examples
            pos_example = metadata[metadata.playlist == q_track.playlist].sample(1)
            neg_example = metadata[metadata.playlist != q_track.playlist].sample(1)

            # create feature pairs
            data_pairs.append([feature_data[[row_id]][0], feature_data[[pos_example.index]][0]])
            labels.append(playlist_similarities.loc[q_track.playlist, pos_example.playlist])

            data_pairs.append([feature_data[[row_id]][0], feature_data[[neg_example.index]][0]])
            labels.append(playlist_similarities.loc[q_track.playlist, neg_example.playlist])

    return np.array(data_pairs), np.array(labels)

In [236]:
data_pairs, labels = create_pairs_with_sims(feature_data, metadata, 10, playlist_similarities)

In [237]:
data_pairs.shape

(18880, 2, 69)

In [238]:
model = create_siamese_network(data_pairs[:,0].shape[1:])

In [239]:
model.fit([data_pairs[:, 0], data_pairs[:, 1]], labels, batch_size=24, verbose=1, epochs=25)

Epoch 1/25
18880/18880 [==============================] - 0s - loss: 0.1089 - mean_squared_error: 0.1089 - acc: 0.7477     
Epoch 2/25
18880/18880 [==============================] - 0s - loss: 0.0701 - mean_squared_error: 0.0701 - acc: 0.8034     
Epoch 3/25
18880/18880 [==============================] - 0s - loss: 0.0566 - mean_squared_error: 0.0566 - acc: 0.8189     
Epoch 4/25
18880/18880 [==============================] - 0s - loss: 0.0476 - mean_squared_error: 0.0476 - acc: 0.8310     
Epoch 5/25
18880/18880 [==============================] - 0s - loss: 0.0399 - mean_squared_error: 0.0399 - acc: 0.8417     
Epoch 6/25
18880/18880 [==============================] - 0s - loss: 0.0347 - mean_squared_error: 0.0347 - acc: 0.8470     
Epoch 7/25
18880/18880 [==============================] - 0s - loss: 0.0300 - mean_squared_error: 0.0300 - acc: 0.8524     
Epoch 8/25
18880/18880 [==============================] - 0s - loss: 0.0270 - mean_squared_error: 0.0270 - acc: 0.8544     
Epoch 9/

In [240]:
evaluate(lambda x,y: [model.predict([x[[i]], y]) for i in range(feature_data.shape[0])], 20)

0.83548728813559325

In [246]:
def create_pairs_with_sims_and_identity(feature_data, metadata, num_pairs_per_track, playlist_similarities):
    
    data_pairs = []
    labels     = []
    
    for row_id, q_track in metadata.sample(frac=1).iterrows():
        
        data_pairs.append([feature_data[[row_id]][0], feature_data[[row_id]][0]])
        labels.append(1)
        
        for _ in range(num_pairs_per_track):
            
            # search similar and dissimilar examples
            pos_example = metadata[metadata.playlist == q_track.playlist].sample(1)
            neg_example = metadata[metadata.playlist != q_track.playlist].sample(1)

            # create feature pairs
            data_pairs.append([feature_data[[row_id]][0], feature_data[[pos_example.index]][0]])
            labels.append(playlist_similarities.loc[q_track.playlist, pos_example.playlist] - 0.1)

            data_pairs.append([feature_data[[row_id]][0], feature_data[[neg_example.index]][0]])
            labels.append(playlist_similarities.loc[q_track.playlist, neg_example.playlist] - 0.1)

    return np.array(data_pairs), np.array(labels)

In [247]:
data_pairs, labels = create_pairs_with_sims_and_identity(feature_data, metadata, 10, playlist_similarities)

In [248]:
model = create_siamese_network(data_pairs[:,0].shape[1:])

In [249]:
model.fit([data_pairs[:, 0], data_pairs[:, 1]], labels, batch_size=24, verbose=1, epochs=25)

Epoch 1/25
19824/19824 [==============================] - 0s - loss: 0.1081 - mean_squared_error: 0.1081 - acc: 0.0517     
Epoch 2/25
19824/19824 [==============================] - 0s - loss: 0.0698 - mean_squared_error: 0.0698 - acc: 0.0551     
Epoch 3/25
19824/19824 [==============================] - 0s - loss: 0.0567 - mean_squared_error: 0.0567 - acc: 0.0564     
Epoch 4/25
19824/19824 [==============================] - 0s - loss: 0.0467 - mean_squared_error: 0.0467 - acc: 0.0570     
Epoch 5/25
19824/19824 [==============================] - 0s - loss: 0.0398 - mean_squared_error: 0.0398 - acc: 0.0572     
Epoch 6/25
19824/19824 [==============================] - 0s - loss: 0.0342 - mean_squared_error: 0.0342 - acc: 0.0575     
Epoch 7/25
19824/19824 [==============================] - 0s - loss: 0.0299 - mean_squared_error: 0.0299 - acc: 0.0578     
Epoch 8/25
19824/19824 [==============================] - 0s - loss: 0.0259 - mean_squared_error: 0.0259 - acc: 0.0578     
Epoch 9/

In [250]:
evaluate(lambda x,y: [model.predict([x[[i]], y]) for i in range(feature_data.shape[0])], 20)

0.84655720338983054

In [286]:
def aggregate_features_sequential(seq_data, track_data, len_segment, m_data, with_year=False, with_popularity=False):
    
    # sequential data
    segments = seq_data["segments"]
    sl       = len(segments)
    
    mfcc              = np.array([s["timbre"]            for s in segments])
    chroma            = np.array([s["pitches"]           for s in segments])
    loudness_max      = np.array([s["loudness_max"]      for s in segments]).reshape((sl,1))
    loudness_start    = np.array([s["loudness_start"]    for s in segments]).reshape((sl,1))
    loudness_max_time = np.array([s["loudness_max_time"] for s in segments]).reshape((sl,1))
    duration          = np.array([s["duration"]          for s in segments]).reshape((sl,1))
    confidence        = np.array([s["confidence"]        for s in segments]).reshape((sl,1))
    
    # concatenate sequential features
    sequential_features = np.concatenate([mfcc, chroma, loudness_max, loudness_start, 
                                          loudness_max_time, duration, confidence], axis=1)
    
    offset  = np.random.randint(0, sl - len_segment)
    segment = sequential_features[offset:(offset+len_segment),:]
        
    # track-based data
    track_features = [track_data[0]["acousticness"],     # acoustic or not?
                      track_data[0]["danceability"],     # danceable?
                      track_data[0]["energy"],           # energetic or calm?
                      track_data[0]["instrumentalness"], # is somebody singing?
                      track_data[0]["liveness"],         # live or studio?
                      track_data[0]["speechiness"],      # rap or singing?
                      track_data[0]["tempo"],            # slow or fast?
                      track_data[0]["time_signature"],   # 3/4, 4/4, 6/8, etc.
                      track_data[0]["valence"]]          # happy or sad?
    
    if with_year:
        track_features.append(int(m_data["year"]))
        
    if with_popularity:
        track_features.append(int(m_data["popularity"]))
        
    
    return segment, track_features


In [287]:
len_segment = 20

sequential_features = []
trackbased_features = []

for i, (_, spotify_data) in enumerate(raw_track_data):
    
    _, _, _, f_sequential, f_trackbased = spotify_data
    
    seq_feat, track_feat = aggregate_features_sequential(f_sequential, 
                                                         f_trackbased, 
                                                         len_segment, 
                                                         metadata.loc[i],
                                                         with_year=True,
                                                         with_popularity=True)
    
    sequential_features.append(seq_feat)
    trackbased_features.append(track_feat)
    
sequential_features = np.asarray(sequential_features)
trackbased_features = np.asarray(trackbased_features)

print "sequential_features.shape:", sequential_features.shape
print "trackbased_features.shape:", trackbased_features.shape

sequential_features.shape: (944, 20, 29)
trackbased_features.shape: (944, 11)


In [301]:
# standardize sequential_features
rows, x, y = sequential_features.shape
sequential_features = sequential_features.reshape(rows, (x * y))
sequential_features -= sequential_features.mean(axis=0)
sequential_features /= sequential_features.std(axis=0)
sequential_features = sequential_features.reshape(rows, x, y)

In [302]:
# standardize trackbased_features
trackbased_features -= trackbased_features.mean(axis=0)
trackbased_features /= trackbased_features.std(axis=0)

In [303]:
def create_pairs_with_sims_and_identity_segments(sequential_features, trackbased_features, metadata, num_pairs_per_track, playlist_similarities):
    
    data_pairs_seq   = []
    data_pairs_track = []
    labels           = []
    
    for row_id, q_track in metadata.sample(frac=1).iterrows():
        
        data_pairs_seq.append([sequential_features[[row_id]][0], sequential_features[[row_id]][0]])
        data_pairs_track.append([trackbased_features[[row_id]][0], trackbased_features[[row_id]][0]])
        labels.append(1)
        
        for _ in range(num_pairs_per_track):
            
            # search similar and dissimilar examples
            pos_example = metadata[metadata.playlist == q_track.playlist].sample(1)
            neg_example = metadata[metadata.playlist != q_track.playlist].sample(1)

            # create feature pairs
            data_pairs_seq.append([sequential_features[[row_id]][0], sequential_features[[pos_example.index]][0]])
            data_pairs_track.append([trackbased_features[[row_id]][0], trackbased_features[[pos_example.index]][0]])
            labels.append(playlist_similarities.loc[q_track.playlist, pos_example.playlist] - 0.1)

            data_pairs_seq.append([sequential_features[[row_id]][0], sequential_features[[neg_example.index]][0]])
            data_pairs_track.append([trackbased_features[[row_id]][0], trackbased_features[[neg_example.index]][0]])
            labels.append(playlist_similarities.loc[q_track.playlist, neg_example.playlist] - 0.1)

    return np.array(data_pairs_seq), np.array(data_pairs_track), np.asarray(labels)

In [304]:
data_pairs_seq, data_pairs_track, labels = create_pairs_with_sims_and_identity_segments(sequential_features,
                                                                                        trackbased_features,
                                                                                        metadata, 
                                                                                        10, 
                                                                                        playlist_similarities)

In [294]:
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional, Input, Lambda
import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda,Convolution1D
from keras.optimizers import RMSprop, Nadam, SGD
from keras.regularizers import l2, l1
from keras import backend as K
#from keras.constraint import unit_norm
from keras.layers.merge import concatenate

In [305]:
input_dim = data_pairs_seq[:, 0].shape[1:]

input_a = Input(shape=data_pairs_seq[:, 0].shape[1:])
input_b = Input(shape=data_pairs_seq[:, 0].shape[1:])
input_a2 = Input(shape=data_pairs_track[:, 0].shape[1:])
input_b2 = Input(shape=data_pairs_track[:, 0].shape[1:])

bdlstm = Bidirectional(LSTM(29, return_sequences=False, activation="selu"))

processed_a = bdlstm(input_a)
processed_b = bdlstm(input_b)

dens = Dense(9, activation="selu")

processed_a2 = dens(input_a2)
processed_b2 = dens(input_b2)

left = concatenate([processed_a, processed_a2], axis=1)
right = concatenate([processed_b, processed_b2], axis=1)

L1_distance = lambda x: K.abs(x[0]-x[1])

distance = Lambda(L1_distance,
                  output_shape=lambda x: x[0])([left, right])

prediction = Dense(29 + 9, activation="elu")(distance)
#prediction = Dense(64, activation="elu")(prediction)

prediction = Dense(1, activation="sigmoid")(prediction)

model = Model([input_a, input_b, input_a2, input_b2], prediction)

# train
rms = Nadam(lr=0.001)
model.compile(loss="mean_squared_error", optimizer=rms, metrics=["mean_squared_error", "accuracy"])

In [306]:
model.fit([data_pairs_seq[:, 0], data_pairs_seq[:, 1], data_pairs_track[:,0], data_pairs_track[:,1]], labels, batch_size=24, verbose=1, epochs=25)

Epoch 1/25
19824/19824 [==============================] - 19s - loss: 0.1451 - mean_squared_error: 0.1451 - acc: 0.0522    
Epoch 2/25
19824/19824 [==============================] - 19s - loss: 0.1023 - mean_squared_error: 0.1023 - acc: 0.0535    
Epoch 3/25
19824/19824 [==============================] - 19s - loss: 0.0857 - mean_squared_error: 0.0857 - acc: 0.0543    
Epoch 4/25
19824/19824 [==============================] - 19s - loss: 0.0712 - mean_squared_error: 0.0712 - acc: 0.0558    
Epoch 5/25
19824/19824 [==============================] - 20s - loss: 0.0595 - mean_squared_error: 0.0595 - acc: 0.0566    
Epoch 6/25
19824/19824 [==============================] - 20s - loss: 0.0511 - mean_squared_error: 0.0511 - acc: 0.0571    
Epoch 7/25
19824/19824 [==============================] - 19s - loss: 0.0446 - mean_squared_error: 0.0446 - acc: 0.0574    
Epoch 8/25
19824/19824 [==============================] - 19s - loss: 0.0386 - mean_squared_error: 0.0386 - acc: 0.0577    
Epoch 9/

In [314]:
def evaluate(similarity_function, cut_off):

    all_precisions = []
    
    pbar = progressbar.ProgressBar()

    for idx in pbar(metadata.index.values):

        dist           = similarity_function(sequential_features, sequential_features[[idx]], trackbased_features, trackbased_features[[idx]])
        dist           = np.array(dist).reshape(len(dist))
        similar_tracks = metadata.loc[np.argsort(dist)[::-1][:cut_off]]
        same_label     = similar_tracks["playlist"] == metadata.loc[idx, "playlist"]
        precision      = same_label.sum() / float(cut_off)
        all_precisions.append(precision)

    all_precisions = np.array(all_precisions)

    return all_precisions.mean()

In [315]:
evaluate(lambda w,x,y,z: [model.predict([w[[i]],x,y[[i]],z]) for i in range(sequential_features.shape[0])], 20)

100% (944 of 944) |#####################################################################################################################################################################################| Elapsed Time: 1:20:00 Time: 1:20:00


0.85317796610169494